<a href="https://colab.research.google.com/github/HrishiDhondge/NVIDIA_workshop/blob/main/Nemo_agent_toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook with Pre-requisites for NVIDIA workshop

In [1]:
# Download and install git-lfs
!wget https://github.com/git-lfs/git-lfs/releases/download/v3.7.0/git-lfs-linux-amd64-v3.7.0.tar.gz

# Extract the files
!tar -xvf git-lfs-linux-amd64-v3.7.0.tar.gz

# Install
!git-lfs-3.7.0/./install.sh

--2025-07-31 02:31:03--  https://github.com/git-lfs/git-lfs/releases/download/v3.7.0/git-lfs-linux-amd64-v3.7.0.tar.gz
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/13021798/7c23ad92-6754-4126-bc83-f9134c4184d4?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-31T03%3A16%3A19Z&rscd=attachment%3B+filename%3Dgit-lfs-linux-amd64-v3.7.0.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-31T02%3A15%3A23Z&ske=2025-07-31T03%3A16%3A19Z&sks=b&skv=2018-11-09&sig=TPcgBhDoUY%2BCbTJMpbeGrFK95ibUG2iZ3hFy0m6iaMM%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1MzkyOTM2MywibmJmIjoxNzUzOTI5MDYzLCJwYXRoIjoic

In [2]:
# Install uv with one of these two ways
#curl -LsSf https://astral.sh/uv/install.sh | sh
!pip install uv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 76.7 MB/s eta 0:00:00


In [20]:
# Clone repo
!git clone https://github.com/NVIDIA/NeMo-Agent-Toolkit.git aiqtoolkit
%cd aiqtoolkit

Cloning into 'aiqtoolkit'...
remote: Enumerating objects: 8528, done.
remote: Counting objects: 100% (327/327), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 8528 (delta 198), reused 109 (delta 101), pack-reused 8201 (from 3)
Receiving objects: 100% (8528/8528), 37.14 MiB | 25.70 MiB/s, done.
Resolving deltas: 100% (4302/4302), done.
Filtering content: 100% (64/64), 20.78 MiB | 11.17 MiB/s, done.
/content/aiqtoolkit


In [21]:
#Initialize, fetch, and update submodules
!git submodule update --init --recursive

Submodule 'external/aiqtoolkit-opensource-ui' (https://github.com/NVIDIA/NeMo-Agent-Toolkit-UI.git) registered for path 'external/aiqtoolkit-opensource-ui'
Cloning into '/content/aiqtoolkit/external/aiqtoolkit-opensource-ui'...
Submodule path 'external/aiqtoolkit-opensource-ui': checked out 'eac58dbe62200729e7bde0ca15cc27efca7b082f'


In [22]:
# Fetch the data sets
!git lfs install
!git lfs fetch
!git lfs pull

Updated Git hooks.
Git LFS initialized.
Fetching reference refs/heads/develop


In [23]:
# Creating the environment
!uv venv --seed .venv --python 3.11 -c
!source .venv/bin/activate

Using CPython 3.11.13 interpreter at: /usr/bin/python3
The following `requires-python` declarations do not permit this version:
- aiqtoolkit-mysql: >=3.12
- aiqtoolkit-s3: >=3.12
Creating virtual environment with seed packages at: .venv
 + pip==25.2
 + setuptools==80.9.0
 + wheel==0.45.1
Activate with: source .venv/bin/activate


In [24]:
# UV syncronization
!uv sync --all-groups --all-extras

Streaming output truncated to the last 5000 lines.
   Building aiq-multi-frameworks @ file:///content/aiqtoolkit/examples/frameworks/mu
   Building aiq-plot-charts @ file:///content/aiqtoolkit/examples/custom_functions/p
   Building aiq-por-to-jiratickets @ file:///content/aiqtoolkit/examples/HITL/por_to
   Building aiq-profiler-agent @ file:///content/aiqtoolkit/examples/advanced_agents
   Building aiqtoolkit-zep-cloud @ file:///content/aiqtoolkit/packages/aiqtoolkit_ze
   Building aiqtoolkit-agno @ file:///content/aiqtoolkit/packages/aiqtoolkit_agno[0
   Building aiqtoolkit-crewai @ file:///content/aiqtoolkit/packages/aiqtoolkit_crewa
   Building aiq-agno-personal-finance @ file:///content/aiqtoolkit/examples/framewor
   Building aiq-alert-triage-agent @ file:///content/aiqtoolkit/examples/advanced_ag
   Building aiq-automated-description-generation @ file:///content/aiqtoolkit/exampl
   Building aiq-email-phishing-analyzer @ file:///content/aiqtoolkit/examples/evalua
   Building ai

In [27]:
# Installing individual plugins
!uv pip install -e '.[langchain]'
!uv pip install -e '.[llama-index]'

# Dependencies for profiling
!uv pip install -e '.[profiling]'

import os
os.environ['aiq'] = './.venv/bin/aiq'
!$aiq --version


Using Python 3.12.11 environment at: /root/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu
error: The interpreter at /root/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu is externally managed, and indicates the following:

  This Python installation is managed by uv and should not be modified.

Consider creating a virtual environment with `uv venv`.
Using Python 3.12.11 environment at: /root/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu
error: The interpreter at /root/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu is externally managed, and indicates the following:

  This Python installation is managed by uv and should not be modified.

Consider creating a virtual environment with `uv venv`.
Using Python 3.12.11 environment at: /root/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu
error: The interpreter at /root/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu is externally managed, and indicates the following:

  This Python installa

In [31]:
!export NVIDIA_API_KEY=<your_api_key>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `export NVIDIA_API_KEY=<your_api_key>'


In [29]:
filename='workflow.yaml'
content = """
functions:
   # Add a tool to search wikipedia
   wikipedia_search:
      _type: wiki_search
      max_results: 2

llms:
   # Tell NeMo Agent toolkit which LLM to use for the agent
   nim_llm:
      _type: nim
      model_name: meta/llama-3.1-70b-instruct
      temperature: 0.0

workflow:
   # Use an agent that 'reasons' and 'acts'
   _type: react_agent
   # Give it access to our wikipedia search tool
   tool_names: [wikipedia_search]
   # Tell it which LLM to use
   llm_name: nim_llm
   # Make it verbose
   verbose: true
   # Retry up to 3 times
   parse_agent_response_max_retries: 3
"""
# Writing to the file
with open(filename, "w") as f:
    f.write(content)

In [30]:
!$aiq run --config_file workflow.yaml --input "List five subspecies of Aardvarks"

2025-07-31 03:08:53,696 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (1898.738384 ms). Ensure all imports are inside your registered functions.
2025-07-31 03:08:53,847 - aiq.runtime.loader - WARNING - Loading module 'aiq.plugins.agno.register' from entry point 'aiq_agno' took a long time (150.740862 ms). Ensure all imports are inside your registered functions.
2025-07-31 03:08:53,866 - aiq.data_models.discovery_metadata - WARNING - Package metadata not found for simple_auth
2025-07-31 03:08:54,025 - aiq.runtime.loader - WARNING - Loading module 'aiq.test.register' from entry point 'aiqtoolkit-test' took a long time (103.553772 ms). Ensure all imports are inside your registered functions.
2025-07-31 03:08:55,835 - aiq.runtime.loader - WARNING - Loading module 'aiq_alert_triage_agent.register' from entry point 'aiq_alert_triage_agent' took a long time (1778.220892 ms). Ensure all imports are inside yo